变长码最明显的缺点在于对差错的敏感性，1位出错可能导致解码器失去同步。

基于某个预先假定的概率模型设计出最佳变长码，可以使之在与信源符号真实概率模型失配不多的前提下简化最佳变长编码器的设计，同时提高解码器可靠性。

__一元码__

非负整数n的一元码位n-1个1(0)后跟1个0(1)，是一种特殊的霍夫曼码

In [33]:
function unarycode(n::Integer, sym::Integer)
    return string(repeat(string(sym), n-1), string(1-sym))
end
[unarycode(n, 1) for n = 1:9]

9-element Vector{String}:
 "0"
 "10"
 "110"
 "1110"
 "11110"
 "111110"
 "1111110"
 "11111110"
 "111111110"

__哥伦布编码__

无须使用霍夫曼算法即可直接给出最佳变长码，使服从几何分布的正整数数据流的平均码长最短，但需要给出满足条件的b值。

条件为:(1-p)^b+(1-p)^(b+1)<=1<=(1-p)^(b-1)+(1-p)^b,0<=p<=1

数据流中整数n出现的概率为P(n) = (1-p)^(n-1)*p, 0<=p<=1

n的哥伦布码由前缀码+尾码组成，前缀码是q+1位的一元码字，q=INT((n-1)/b)，尾码是(n-1)/b的余数r=n-qb的二进制编码且长为INT(log_2(b)+0.5)

另一参考编码方法[Link](https://zhuanlan.zhihu.com/p/180520059)

In [79]:
# 该过程得到的结果与书上不同，因为不明白书上的尾码二进制编码方法
function golombcode(n, b)
    q = (n-1) ÷ b
    r = (n-1) % b
    c = Integer(floor(log2(b)))
    if r < 2^c-b
        str = string(r, base=2, pad=c-1)
    else
        str = string(r, base=2, pad=c)
    end
    return string(unarycode(q+1, 0), str)
end
for b=3:5
    println([golombcode(n, b) for n = 1:10])
end

["10", "11", "110", "010", "011", "0110", "0010", "0011", "00110", "00010"]
["100", "101", "110", "111", "0100", "0101", "0110", "0111", "00100", "00101"]
["100", "101", "110", "111", "1100", "0100", "0101", "0110", "0111", "01100"]


当b=2^k时，可使编码过程简化，此时n的码字构成位：尾码用n的二进制表示的最低k位，前缀码位n的二进制表示的其余高位+1的一元码字。

这类特殊的哥伦布码记为G(k)。如果k=0,b=1没有尾码，则G(0)为一元码，可以看出：一元码是哥伦布码的特例，G(k)又是一元码作为前缀的截断霍夫曼码

In [81]:
function golombcodeg(n, b)
    q = n ÷ b
    r = n % b
    c = Integer(floor(log2(b)))
    return string(unarycode(q+1, 0), string(r, base=2, pad=c))
end
println([golombcodeg(n, 4) for n = 1:10])

["101", "110", "111", "0100", "0101", "0110", "0111", "00100", "00101", "00110"]


指数哥伦布编码(EGC)：G(0)码加q+m位尾码

通用变长码(UVLC)：前后缀交织的0阶EGC